In [1]:
#meta 3/15/2022 Load CSV and Numpy File Types in TensorFlow 2.0
# src course Introduction to TensorFlow
# git clone https://github.com/GoogleCloudPlatform/training-data-analyst 
# file src: https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/labs/load_diff_filedata.ipynb

#infra: work laptop, env anya_tf2

#In the notebook interface, navigate to 
#  training-data-analyst > courses > machine_learning > deepdive2 > introduction_to_tensorflow > labs, 
#  and open load_diff_filedata.ipynb
#Look at the complete solution, navigate to 
#  training-data-analyst > courses > machine_learning > deepdive2 > introduction_to_tensorflow > solutions, 
#  and open load_diff_filedata.ipynb

#history
#3/15/2022 REVIEW
#      Download data files, preview data
#      1) Read CSV files into a tf dataset
#      Data preprocessing:
#        Continious data (aka numeric), w/ normalization
#        Categorical data
#        Combined preprocessing layer
#      2) Load NumPy data into a tf dataset
#
#      myXtra Doc: functools.partial()

# Reference:
#$error
#  Jupyter notebook: 'head' is not recognized as an internal or external command, operable program or batch file.
#$fix: 
#  https://stackoverflow.com/questions/61748573/jupyter-notebook-head-is-not-recognized-as-an-internal-or-external-command

# How to Load CSV and Numpy File Types in TensorFlow 2.0



## Learning Objectives

1. Load a CSV file into a `tf.data.Dataset`. 
2. Load Numpy data




## Introduction 

In this lab, you load CSV data from a file into a `tf.data.Dataset`.  This tutorial provides an example of loading data from NumPy arrays into a `tf.data.Dataset` you also load text data.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review [the solution notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/solutions/load_diff_filedata.ipynb).

## Load necessary libraries 
We will start by importing the necessary libraries for this lab.

In [2]:
import functools

import numpy as np
import tensorflow as tf

print("TensorFlow version: ",tf.version.VERSION)

TensorFlow version:  2.6.3


In [3]:
#$my get data files
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

#$myNote: downloaded file into C:\Users\chq-anyac\.keras\datasets
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [4]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

## Load data

This section provides an example of how to load CSV data from a file into a `tf.data.Dataset`.  The data used in this tutorial are taken from the Titanic passenger list. The model will predict the likelihood a passenger survived based on characteristics like age, gender, ticket class, and whether the person was traveling alone.

To start, let's look at the top of the CSV file to see how it is formatted.

In [5]:
#$my - works in Linux/Unix system, ie. G Colab
# !head {train_file_path}

'head' is not recognized as an internal or external command,
operable program or batch file.


$error  
Jupyter notebook: 'head' is not recognized as an internal or external command, operable program or batch file.

In [6]:
#$fix: https://stackoverflow.com/questions/61748573/jupyter-notebook-head-is-not-recognized-as-an-internal-or-external-command

#$my - to make it work in Windows
my_train_file_path = 'data/train.csv'
n_lines = 5

#option 1
#read # lines in a file
with open(my_train_file_path) as f:
    for _ in range(n_lines): # first n lines
        print(f.readline())

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone

0,male,22.0,1,0,7.25,Third,unknown,Southampton,n

1,female,38.0,1,0,71.2833,First,C,Cherbourg,n

1,female,26.0,0,0,7.925,Third,unknown,Southampton,y

1,female,35.0,1,0,53.1,First,C,Southampton,n



In [7]:
#option 2
%alias head powershell -command "& {Get-Content %s -Head 5}"
%head {my_train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n


You can [load this using pandas](pandas_dataframe.ipynb), and pass the NumPy arrays to TensorFlow. If you need to scale up to a large set of files, or need a loader that integrates with [TensorFlow and tf.data](../../guide/data.ipynb) then use the `tf.data.experimental.make_csv_dataset` function:

The only column you need to identify explicitly is the one with the value that the model is intended to predict. 

In [8]:
# TODO 1: Add string name for label column 
LABEL_COLUMN = 'survived' 
LABELS = [0,1]

Now read the CSV data from the file and create a dataset. 

(For the full documentation, see `tf.data.experimental.make_csv_dataset`)  
https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset

Reads CSV files into a dataset, where each element of the dataset is a (features, labels) tuple that corresponds to a batch of CSV rows. The features dictionary maps feature column names to `Tensors` containing the corresponding feature data, and labels is a `Tensor` containing the batch's label data.  More detail in part `Xtra`.


In [9]:
def get_dataset(file_path, **kwargs):
# TODO 2
# TODO: Read the CSV data from the file and create a dataset 
    dataset = tf.data.experimental.make_csv_dataset( 
    file_path,
          batch_size=5, # Artificially small to make examples easier to show.
          label_name=LABEL_COLUMN,
          na_value="?",
          num_epochs=1,
          ignore_errors=False, #$my was True
          **kwargs
    ) 
    return dataset


In [10]:
#$my create datasets
raw_train_data = get_dataset(train_file_path)# TODO: Your code goes here.
raw_test_data = get_dataset(test_file_path)# TODO: Your code goes here.

#$my ds type
raw_train_data.__class__ #class tensorflow.python.data.ops.dataset_ops.PrefetchDataset

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [11]:
#$my show_batch() equivalent
for feat, targ in raw_train_data.take(1):
    print ('Features:')
    for key, value in feat.items():
      print("{:20s}: {}".format(key,value.numpy()))
    print ('Labels: {}'.format(targ))

Features:
sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [20. 28. 20. 25. 31.]
n_siblings_spouses  : [1 0 1 1 0]
parch               : [0 0 1 0 0]
fare                : [ 7.925  7.896 15.742 26.    10.5  ]
class               : [b'Third' b'Third' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'n' b'n' b'y']
Labels: [1 0 1 0 0]


In [12]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Each item in the dataset is a batch, represented as a tuple of (*many examples*, *many labels*). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (5 in this case).

It might help to see this yourself.

In [13]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'female']
age                 : [22. 30. 15. 19. 30.]
n_siblings_spouses  : [0 3 1 0 0]
parch               : [0 0 1 0 0]
fare                : [10.517 21.     7.229  7.775 56.929]
class               : [b'Third' b'Second' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg']
alone               : [b'y' b'n' b'n' b'y' b'y']


As you can see, the columns in the CSV are named. The dataset constructor will pick these names up automatically. If the file you are working with does not contain the column names in the first line, pass them in a list of strings to  the `column_names` argument in the `make_csv_dataset` function.

In [14]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'female' b'female']
age                 : [36. 51. 32. 28. 60.]
n_siblings_spouses  : [1 0 0 0 1]
parch               : [2 0 0 0 0]
fare                : [120.      7.054   8.05    7.879  75.25 ]
class               : [b'First' b'Third' b'Third' b'Third' b'First']
deck                : [b'B' b'unknown' b'E' b'unknown' b'D']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'n']


This example is going to use all the available columns. If you need to omit some columns from the dataset, create a list of just the columns you plan to use, and pass it into the (optional) `select_columns` argument of the constructor.


In [15]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 49. 29. 17. 42.]
n_siblings_spouses  : [0 1 1 1 1]
class               : [b'Second' b'First' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'n' b'n' b'n']


## Data preprocessing

A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model.

TensorFlow has a built-in system for describing common input conversions: `tf.feature_column`, see [this tutorial](https://www.tensorflow.org/tutorials/structured_data/feature_columns) for details.  
https://www.tensorflow.org/tutorials/structured_data/feature_columns


You can preprocess your data using any tool you like (like [nltk](https://www.nltk.org/) or [sklearn](https://scikit-learn.org/stable/)), and just pass the processed output to TensorFlow. 


The primary advantage of doing the preprocessing inside your model is that when you export the model it includes the preprocessing. This way you can pass the raw data directly to your model.

### Continuous data

If your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:

In [16]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [50. 24. 29. 24. 28.]
n_siblings_spouses  : [2. 0. 1. 0. 0.]
parch               : [0. 0. 1. 0. 0.]
fare                : [133.65   13.     10.462  69.3    52.   ]


In [17]:
example_batch, labels_batch = next(iter(temp_dataset)) 

#$my preview
example_batch, labels_batch

(OrderedDict([('age',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([50., 44., 58., 30., 28.], dtype=float32)>),
              ('n_siblings_spouses',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 0., 0., 0.], dtype=float32)>),
              ('parch',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 0., 1., 0., 0.], dtype=float32)>),
              ('fare',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 26.   ,  26.   , 153.462,   7.896,   7.896], dtype=float32)>)]),
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 0, 1, 0, 0])>)

Here's a simple function that will pack together all the columns:

In [18]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

Apply this to each element of the dataset:

In [19]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[30.     0.     0.     8.663]
 [24.     1.     0.    16.1  ]
 [26.     1.     1.    26.   ]
 [17.     0.     0.     8.663]
 [ 7.     0.     2.    26.25 ]]

[0 0 0 0 1]


If you have mixed datatypes you may want to separate out these simple-numeric fields. The `tf.feature_column` api can handle them, but this incurs some overhead and should be avoided unless really necessary. Switch back to the mixed dataset:

In [20]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'female' b'female']
age                 : [40.5 40.  32.   2.  17. ]
n_siblings_spouses  : [0 0 0 0 4]
parch               : [0 0 0 1 2]
fare                : [ 7.75  13.    13.    10.462  7.925]
class               : [b'Third' b'Second' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'G' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'n']


In [21]:
example_batch, labels_batch = next(iter(temp_dataset)) 

#$my preview
example_batch, labels_batch

(OrderedDict([('age',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([31., 28., 18., 27., 46.], dtype=float32)>),
              ('n_siblings_spouses',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 1.], dtype=float32)>),
              ('parch',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 2., 0., 0.], dtype=float32)>),
              ('fare',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([13.   ,  6.95 , 13.   ,  7.896, 61.175], dtype=float32)>)]),
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 0, 1, 0, 0])>)

So define a more general preprocessor that selects a list of numeric features and packs them into a single column:

In [22]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [23]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [24]:
show_batch(packed_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
class               : [b'First' b'Third' b'First' b'Third' b'Third']
deck                : [b'B' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']
numeric             : [[44.     0.     0.    27.721]
 [28.     0.     0.     7.787]
 [45.     0.     0.    35.5  ]
 [28.     8.     2.    69.55 ]
 [16.     0.     0.     8.05 ]]


In [25]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#$my preview
example_batch, labels_batch

(OrderedDict([('sex',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'female', b'female', b'female', b'male', b'male'], dtype=object)>),
              ('class',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Third', b'Second', b'Third', b'First', b'First'], dtype=object)>),
              ('deck',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'unknown', b'unknown', b'unknown', b'unknown', b'B'], dtype=object)>),
              ('embark_town',
               <tf.Tensor: shape=(5,), dtype=string, numpy=
               array([b'Cherbourg', b'Southampton', b'Cherbourg', b'Southampton',
                      b'Cherbourg'], dtype=object)>),
              ('alone',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'n', b'n', b'n', b'y', b'y'], dtype=object)>),
              ('numeric',
               <tf.Tensor: shape=(5, 4), dtype=float32, numpy=
               array([[14.   ,  1.   ,  0.   , 11.242],
              

#### Data Normalization

Continuous data should always be normalized.

In [28]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [29]:
#$my 
desc.T

,count,mean,std,min,25%,50%,75%,max
age,627.0,29.631308,12.511818,0.75,23.0000,28.0000,35.0000,80.0000
n_siblings_spouses,627.0,0.545455,1.151090,0.00,0.0000,0.0000,1.0000,8.0000
parch,627.0,0.379585,0.792999,0.00,0.0000,0.0000,0.0000,5.0000
fare,627.0,34.385399,54.597730,0.00,7.8958,15.0458,31.3875,512.3292


In [30]:
# TODO 1
MEAN = np.array(desc.T['mean'])# TODO: Your code goes here.
STD = np.array(desc.T['std'])# TODO: Your code goes here.

print(MEAN, STD)

[29.631  0.545  0.38  34.385] [12.512  1.151  0.793 54.598]


In [31]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  # TODO 2
    return (data-mean)/std

Now create a numeric column.  
https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column  
The `tf.feature_columns.numeric_column` API accepts a `normalizer_fn` argument, which will be run on each batch.

Bind the `MEAN` and `STD` to the normalizer fn using [`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial).

In [32]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000018233DFD9D8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

When you train the model, include this feature column to select and center this block of numeric data:

In [33]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[14.   ,  1.   ,  0.   , 11.242],
       [22.   ,  1.   ,  1.   , 29.   ],
       [ 0.75 ,  2.   ,  1.   , 19.258],
       [28.   ,  0.   ,  0.   , 47.1  ],
       [46.   ,  0.   ,  0.   , 79.2  ]], dtype=float32)>

In [34]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-1.249,  0.395, -0.479, -0.424],
       [-0.61 ,  0.395,  0.782, -0.099],
       [-2.308,  1.264,  0.782, -0.277],
       [-0.13 , -0.474, -0.479,  0.233],
       [ 1.308, -0.474, -0.479,  0.821]], dtype=float32)

The mean based normalization used here requires knowing the means of each column ahead of time.

### Categorical data

Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of options.

Use the `tf.feature_column` API to create a collection with a `tf.feature_column.indicator_column` for each categorical column.  
https://www.tensorflow.org/api_docs/python/tf/feature_column/indicator_column  
https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list


In [35]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [36]:
categorical_columns = [] #list
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [37]:
# See what you just created.
categorical_columns # list of tensorflow.python.feature_column.feature_column_v2.IndicatorColumn

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [60]:
#$my preview
print(categorical_columns[1].name, categorical_columns[1].variable_shape)
print(categorical_columns[2].name, categorical_columns[2].variable_shape)
categorical_columns[1].get_config()

class_indicator (1, 3)
deck_indicator (1, 10)


{'categorical_column': {'class_name': 'VocabularyListCategoricalColumn',
  'config': {'key': 'class',
   'vocabulary_list': ('First', 'Second', 'Third'),
   'dtype': 'string',
   'default_value': -1,
   'num_oov_buckets': 0}}}

In [80]:
#$my preview - how many total features resulted
[print(cat_cols.variable_shape[1]) for cat_cols in categorical_columns];

cat_cols_total = 0
for cat_cols in categorical_columns:
    cat_cols_total = cat_cols_total + cat_cols.variable_shape[1]
    
cat_cols_total

2
3
10
3
2


20

In [61]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])
#$my preview
#print(categorical_layer(example_batch).numpy())

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]


This will be become part of a data processing input later when you build the model.

### Combined preprocessing layer

Add the two feature column collections and pass them to a `tf.keras.layers.DenseFeatures` to create an input layer that will extract and preprocess both input types:

In [82]:
# TODO 1
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns) # TODO: Your code goes here.

In [83]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     1.     0.     0.    -1.249  0.395
 -0.479 -0.424  0.     1.   ]


In [102]:
#$my preview - order of all features, alphabetcial order explains 'numeric' before 'sex_indicator'
preprocessing_layer.__class__ #keras.feature_column.dense_features_v2.DenseFeatures
[print(k.name) for k in categorical_columns];
[print(k.name) for k in numeric_columns];


sex_indicator
class_indicator
deck_indicator
embark_town_indicator
alone_indicator
numeric


### Next Step

A next step would be to build a build a `tf.keras.Sequential`, starting with the `preprocessing_layer`, which is beyond the scope of this lab.  We will cover the Keras Sequential API in the next Lesson.

# Load NumPy data

## Load necessary libraries 
First, restart the Kernel.  Then, we will start by importing the necessary libraries for this lab.

In [103]:
import numpy as np
import tensorflow as tf

print("TensorFlow version: ",tf.version.VERSION)

TensorFlow version:  2.6.3


### Load data from `.npz` file

We use the MNIST dataset in Keras.

In [105]:
#$myNote: downloaded file into C:\Users\chq-anyac\.keras\datasets\mnist.npz (4 .npy files: ie. x_train.npy, x_test.npy)

DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
# TODO 1
  train_examples = data['x_train']# TODO: Your code goes here.
  train_labels = data['y_train'] #TODO: Your code goes here.
  test_examples = data['x_test'] # TODO: Your code goes here.
  test_labels = data['y_test'] # TODO: Your code goes here.

## Load NumPy arrays with `tf.data.Dataset`

Assuming you have an array of examples and a corresponding array of labels, pass the two arrays as a tuple into `tf.data.Dataset.from_tensor_slices` to create a `tf.data.Dataset`.  
https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [110]:
# TODO 2
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels)) # TODO: Your code goes here.
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels)) # TODO: Your code goes here.

In [114]:
for feat, targ in train_dataset.take(1):
    print ('Features: {}'.format(feat)) #class 'tensorflow.python.framework.ops.EagerTensor'
    print ('Labels: {}'.format(targ))


Features: [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 

### Next Step

A next step would be to build a build a `tf.keras.Sequential`, starting with the `preprocessing_layer`, which is beyond the scope of this lab.  We will cover the Keras Sequential API in the next Lesson.

### Resources 
1. Load text data - this link: https://www.tensorflow.org/tutorials/load_data/text
2. TF.text - this link:  https://www.tensorflow.org/tutorials/tensorflow_text/intro
3. Load image daeta - https://www.tensorflow.org/tutorials/load_data/images
4. Read data into a Pandas DataFrame - https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
5. How to represent Unicode strings in TensorFlow - https://www.tensorflow.org/tutorials/load_data/unicode
6. TFRecord and tf.Example -  https://www.tensorflow.org/tutorials/load_data/tfrecord                  

Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Xtra

### Preview a .csv file
$error  
Jupyter notebook: 'head' is not recognized as an internal or external command, operable program or batch file.

https://stackoverflow.com/questions/61748573/jupyter-notebook-head-is-not-recognized-as-an-internal-or-external-command

$my #option 3 - hardcode file path  
or 'data\\train.csv'

In [116]:
%%bash
head 'data/train.csv' 

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


## Doc: tf.data.experimental.make_csv_dataset
Reads CSV files into a dataset

TensorFlow -> API -> TensorFlow Core v2.8.0 -> Python  
https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset

In [117]:
#$xtra doc
doc_filename = 'data/doc_example.csv'

In [118]:
%%bash
head 'data/doc_example.csv' 

Feature_A,Feature_B
1,a
2,b
3,c
4,d

By default, the dataset is repeated indefinitely, reshuffling the order each time. This behavior can be modified by setting the `num_epochs` and `shuffle` arguments.

In [119]:
# No label column specified
dataset = tf.data.experimental.make_csv_dataset(doc_filename, batch_size=2)
#dataset = tf.data.experimental.make_csv_dataset(doc_filename, batch_size=2, shuffle=False)
iterator = dataset.as_numpy_iterator()
print(dict(next(iterator)))

# prints a dictionary of batched features:

{'Feature_A': array([4, 3]), 'Feature_B': array([b'd', b'c'], dtype=object)}


In [120]:
# Set Feature_B as label column
dataset = tf.data.experimental.make_csv_dataset(
    doc_filename, batch_size=2, label_name="Feature_B")
iterator = dataset.as_numpy_iterator()
print(next(iterator))

# prints (features, labels) tuple:

(OrderedDict([('Feature_A', array([2, 3]))]), array([b'b', b'c'], dtype=object))


## Doc: functools.partial()
https://docs.python.org/3/library/functools.html#functools.partial

The `partial()` is used for partial function application which “freezes” some portion of a function’s arguments and/or keywords resulting in a new object with a simplified signature. For example, `partial()` can be used to create a callable that behaves like the `int()` function where the base argument defaults to two:

In [121]:
#$xtra doc
#from functools import partial
basetwo = functools.partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.'
basetwo('10010')

18

In [122]:
#original function int()
#refer to https://docs.python.org/3/library/functions.html#int

#default base=10
print(int("123"), 1*(10**2) + 2*(10**1) + 3*(10**0))
#change base=2
print(int('10010', base=2), 1*(2**4) + 0*(2**3) + 0*(2**2) + 1*(2**1) + 0*(2**0))

123 123
18 18
